In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import matplotlib.pyplot as plt

# reset everything to rerun in jupyter
tf.reset_default_graph()

# read data reques timeout, could download to localhost first.
mnist = input_data.read_data_sets("data/mnist", one_hot=True)

learning_rate = 0.01
batch_size = 128
n_epoches = 25

# input name space for tensorboard.
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, [batch_size, 784], name="image")
    Y = tf.placeholder(tf.float32, [batch_size, 10], name="label")
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

# train variables name space.
with tf.name_scope('weight'):
    w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name="weight")
    
with tf.name_scope('bias'):
    b = tf.Variable(tf.zeros([1, 10]), name="bias")

with tf.name_scope('logits'):
    logits = tf.matmul(X, w) + b

# use softmax cross entropy with logits as the loss function
# compute the mean cross entropy, softmax is applied internally.
with tf.name_scope('entropy'):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
# computes the mean over examples in the batch

with tf.name_scope('loss'):
    loss = tf.reduce_mean(entropy)
    
# can move accuracy into defintion, and put into optimizer in sess.run then.
with tf.name_scope('predict'):
    correct_preds = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

# scalar for tensorboard.
tf.summary.scalar("cost", loss)
tf.summary.scalar('accuracy', accuracy)

summary_op = tf.summary.merge_all()

init = tf.global_variables_initializer()

with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    sess.run(init)
    # training the batch
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epoches):
        avg_cost = 0.
        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, l, summary = sess.run([optimizer, loss, summary_op], feed_dict={X: X_batch, Y:Y_batch})
            
            # Compute average loss
            avg_cost += l / n_batches
            
            # write log
            writer.add_summary(summary, n_epoches * n_batches + i)

    # testing the model
    n_batches = int(mnist.test.num_examples/batch_size)
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        _, acc, summary = sess.run([optimizer, accuracy, summary_op], feed_dict={X: X_batch, Y: Y_batch})
        # write log
        writer.add_summary(summary, n_epoches * n_batches + i)
    

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz
